In [1]:
import torch
import os
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import copy
import time
import pickle

In [2]:
CONFIGS = {
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    # specify ImageNet mean and standard deviation
    "IMG_MEAN": [0.485, 0.456, 0.406],
    "IMG_STD": [0.229, 0.224, 0.225],
    "INIT_LR": 1e-4,
    "NUM_EPOCHS": 200,
    "BATCH_SIZE": 16,
    # specify the loss weights
    "LABELS_PRDTYPE": 1.0,
    "LABELS_WEIGHT": 1.0,
    "LABELS_HALAL": 1.0,
    "LABELS_HEALTHY": 1.0,
    "MODEL_PATH": os.path.sep.join(["output", "detector.pth"]),
    "LE_PATH_PRDTYPE": os.path.sep.join(["output", "le_prdtype.pickle"]),
    "LE_PATH_WEIGHT": os.path.sep.join(["output", "le_weight.pickle"]),
    "LE_PATH_HALAL": os.path.sep.join(["output", "le_halal.pickle"]),
    "LE_PATH_HEALTHY": os.path.sep.join(["output", "le_healthy.pickle"]),
    "PIN_MEMORY": True if torch.cuda.is_available() else False,
    "DATA_BASE_PATH": os.path.join('..', 'rshiny', 'www', 'all_images'),
    "NEW_DATA_BASE_PATH": os.path.join('..', 'small_model', 'new_imgs'),
    "BASE_PATH": os.path.join('..'),
}

# create output folder
if not os.path.exists("output"):
    !mkdir -p {"output"}

In [3]:
prd_group_df = pd.read_csv(os.path.join(CONFIGS['BASE_PATH'], "bayesian_model/product_group.csv"))
# prd_group_df['group'] = pd.to_numeric(prd_group_df['group'], errors='coerce')
prd_group_df['group'] = prd_group_df['group'].fillna(0)
# prd_group_df['tokeep'] = (prd_group_df['group'].astype(int) >= 1) & (prd_group_df['group'].astype(int) <= 7)
prd_group_df['tokeep'] = prd_group_df['group'].astype(int) == 1
prd_group_df.head()

filepath  \
0  IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...   
1  IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...   
2  IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...   
3  IMG_20230428_123521_jpg.rf.1069b402272252862ec...   
4  IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...   

                                label ProductType    Weight HalalStatus  \
0  Sugar_400-499g_NonHalal_NonHealthy       Sugar  400-499g    NonHalal   
1  Sugar_400-499g_NonHalal_NonHealthy       Sugar  400-499g    NonHalal   
2  Sugar_400-499g_NonHalal_NonHealthy       Sugar  400-499g    NonHalal   
3  Sugar_400-499g_NonHalal_NonHealthy       Sugar  400-499g    NonHalal   
4  Sugar_400-499g_NonHalal_NonHealthy       Sugar  400-499g    NonHalal   

  HealthStatus  new_camera  tag  group  tokeep  
0   NonHealthy           0  NaN    1.0    True  
1   NonHealthy           0  NaN    1.0    True  
2   NonHealthy           0  NaN    2.0   False  
3   NonHealthy           0  NaN    1.0    True  
4   NonHealthy           0  NaN    2.0   False

In [4]:
annotations_0 = pd.read_csv("../master_list.csv")
print(annotations_0.shape)
annotations_0 = pd.merge(annotations_0, prd_group_df[["filepath", "tokeep"]], how='left', on='filepath')

annotations_1 = annotations_0.loc[annotations_0["tokeep"]==True]
annotations_1 = annotations_1.groupby('ProductType').apply(lambda x: x.sample(n=min(len(x), 10))).reset_index(drop=True)
print(annotations_1.shape)

annotations_2 = annotations_0[annotations_0['tag'] == 'TestforMode5']
annotations_2.reset_index(drop=True, inplace=True)
print(annotations_2.shape)

# Concatenate the two dataframes vertically
annotations = pd.concat([annotations_1, annotations_2], ignore_index=True)
annotations.reset_index(drop=True, inplace=True)
print(annotations.shape)

(6850, 13)
(662, 14)
(10, 14)
(672, 14)


In [8]:
annotations.to_csv("test.csv")

In [5]:
# Initialize lists for processed data
data, imagePaths, filenames = [], [], []

# Process each annotation entry
for idx, row in annotations.iterrows():
    filepath = row["filepath"]
    imagePath = os.path.join(CONFIGS["DATA_BASE_PATH"], filepath)
    # if row['Type'] == 'old':
    #     imagePath = os.path.join(CONFIGS["DATA_BASE_PATH"], filepath)
    # else:
    #     imagePath = os.path.join("/content", CONFIGS["NEW_DATA_BASE_PATH"], filepath)
    image = cv2.imread(imagePath)
    
    # Preprocess image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (60, 60))

    # Append processed data to lists
    data.append(image)
    imagePaths.append(imagePath)
    # filenames.append(filepath.rsplit('.', 1)[0])
    filenames.append(filepath)

# Convert data to NumPy arrays for machine learning processing
labels = {
    'labels_prdtype': annotations['ProductType'],
    'labels_weight': annotations['Weight'],
    'labels_halal': annotations['HalalStatus'],
    'labels_healthy': annotations['HealthStatus'],
    'labels_total': annotations['label']
}

data = np.array(data, dtype="float32")
for label_name, label_data in labels.items():
    labels[label_name] = np.array(label_data)

# Split the data and labels into training and testing sets
split = train_test_split(data, *labels.values(), imagePaths, filenames,
                         test_size=0.3, random_state=42, stratify=labels['labels_prdtype'])

# Unpack the data split
(trainImages, testImages, *split_labels, trainPaths, testPaths, trainFilenames, testFilenames) = split

# Create label encoders and transform labels
le_prdtype = LabelEncoder()
le_weight = LabelEncoder()
le_halal = LabelEncoder()
le_healthy = LabelEncoder()
le_total = LabelEncoder()

trainLabels = {}
testLabels = {}

# Fit label encoders and transform labels
trainLabels['labels_prdtype'] = le_prdtype.fit_transform(split_labels[0])
testLabels['labels_prdtype'] = le_prdtype.transform(split_labels[1])

trainLabels['labels_weight'] = le_weight.fit_transform(split_labels[2])
testLabels['labels_weight'] = le_weight.transform(split_labels[3])

trainLabels['labels_halal'] = le_halal.fit_transform(split_labels[4])
testLabels['labels_halal'] = le_halal.transform(split_labels[5])

trainLabels['labels_healthy'] = le_healthy.fit_transform(split_labels[6])
testLabels['labels_healthy'] = le_healthy.transform(split_labels[7])

trainLabels['labels_total'] = le_total.fit_transform(split_labels[8])
testLabels['labels_total'] = le_total.transform(split_labels[9])


# Convert NumPy arrays to PyTorch tensors
trainImages, testImages = torch.tensor(trainImages), torch.tensor(testImages)
for label_name in labels.keys():
    trainLabels[label_name] = torch.tensor(trainLabels[label_name])
    testLabels[label_name] = torch.tensor(testLabels[label_name])


ValueError: y contains previously unseen labels: 'BeehoonVermicelli_500-599g_Halal_NonHealthy'

In [ ]:
class CustomTensorDataset(Dataset):
    # Initialize the constructor
    def __init__(self, images, labels, filenames, transforms=None):
        self.images = images
        self.labels = labels
        self.filenames = filenames
        self.transforms = transforms

    def __getitem__(self, index):
        # Grab the image, labels, and its bounding box coordinates
        image = self.images[index]
        label_prdtype = self.labels['labels_prdtype'][index]
        label_weight = self.labels['labels_weight'][index]
        label_halal = self.labels['labels_halal'][index]
        label_healthy = self.labels['labels_healthy'][index]
        filename = self.filenames[index]

        # Transpose the image such that its channel dimension becomes the leading one
        image = image.permute(2, 0, 1)

        # Check to see if we have any image transformations to apply and if so, apply them
        if self.transforms:
            image = self.transforms(image)

        # Return a tuple of the images, labels, and bounding box coordinates
        return (image, label_prdtype, label_weight, label_halal, label_healthy, filename)

    def __len__(self):
        # Return the size of the dataset
        return len(self.images)

# Define normalization and augmentation transforms
normalization_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=CONFIGS['IMG_MEAN'], std=CONFIGS['IMG_STD'])
])

augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(20),
    # transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1)
])

# Combine augmentation and normalization for training
train_transforms = transforms.Compose([augmentation_transforms, normalization_transforms])
test_transforms = normalization_transforms

# Create PyTorch datasets
trainDS = CustomTensorDataset(trainImages, trainLabels, trainFilenames, transforms=train_transforms)
testDS = CustomTensorDataset(testImages, testLabels, testFilenames, transforms=test_transforms)

# Print dataset sizes
print("[INFO] total training samples: {}...".format(len(trainDS)))
print("[INFO] total test samples: {}...".format(len(testDS)))

# Calculate steps per epoch for training and validation set
trainSteps = len(trainDS) // CONFIGS['BATCH_SIZE']
valSteps = len(testDS) // CONFIGS['BATCH_SIZE']

# Create data loaders
# trainLoader = DataLoader(trainDS, batch_size=CONFIGS['BATCH_SIZE'], shuffle=True,
#                          num_workers=os.cpu_count(), pin_memory=CONFIGS['PIN_MEMORY'])
# testLoader = DataLoader(testDS, batch_size=CONFIGS['BATCH_SIZE'],
#                         num_workers=os.cpu_count(), pin_memory=CONFIGS['PIN_MEMORY'])

trainLoader = DataLoader(trainDS, batch_size=CONFIGS['BATCH_SIZE'], shuffle=True, num_workers=0, pin_memory=CONFIGS['PIN_MEMORY'])
testLoader = DataLoader(testDS, batch_size=CONFIGS['BATCH_SIZE'], num_workers=0, pin_memory=CONFIGS['PIN_MEMORY'])

In [ ]:
# Define the MultiHeadResNet model
class MultiHeadResNet(nn.Module):
    def __init__(self, num_classes_prdtype, num_classes_weight, num_classes_halal, num_classes_healthy):
        super(MultiHeadResNet, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()

        # Define custom fully connected layers for each prediction head
        self.fc_prdtype = nn.Linear(num_ftrs, num_classes_prdtype)
        self.fc_weight = nn.Linear(num_ftrs, num_classes_weight)
        self.fc_halal = nn.Linear(num_ftrs, num_classes_halal)
        self.fc_healthy = nn.Linear(num_ftrs, num_classes_healthy)

    def forward(self, x):
        x = self.base_model(x)
        prdtype = self.fc_prdtype(x)
        weight = self.fc_weight(x)
        halal = self.fc_halal(x)
        healthy = self.fc_healthy(x)
        return prdtype, weight, halal, healthy

# Function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    corrects = torch.sum(preds == labels.data)
    return corrects.double() / labels.size(0)

# Training and Validation Loop with Early Stopping
def train_model(model, criteria, optimizer, train_loader, test_loader, device, num_epochs=25, early_stopping_patience=10):
    criterion_prdtype, criterion_weight, criterion_halal, criterion_healthy = criteria
    best_val_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())
    epochs_no_improve = 0

    history = {
        'train_loss': [],
        'train_acc_prdtype': [],
        'train_acc_weight': [],
        'train_acc_halal': [],
        'train_acc_healthy': [],
        'train_acc_overall': [],
        'val_loss': [],
        'val_acc_prdtype': [],
        'val_acc_weight': [],
        'val_acc_halal': [],
        'val_acc_healthy': [],
        'val_acc_overall': [],
    }

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects_prdtype = 0
            running_corrects_weight = 0
            running_corrects_halal = 0
            running_corrects_healthy = 0
            running_corrects_overall = 0
            total_samples = 0

            for inputs, label_prdtype, label_weight, label_halal, label_healthy, _ in train_loader if phase == 'train' else test_loader:
                inputs = inputs.to(device)
                label_prdtype = label_prdtype.to(device)
                label_weight = label_weight.to(device)
                label_halal = label_halal.to(device)
                label_healthy = label_healthy.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs_prdtype, outputs_weight, outputs_halal, outputs_healthy = model(inputs)
                    loss_prdtype = criterion_prdtype(outputs_prdtype, label_prdtype)
                    loss_weight = criterion_weight(outputs_weight, label_weight)
                    loss_halal = criterion_halal(outputs_halal, label_halal)
                    loss_healthy = criterion_healthy(outputs_healthy, label_healthy)
                    loss = loss_prdtype + loss_weight + loss_halal + loss_healthy  # Total loss

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects_prdtype += calculate_accuracy(outputs_prdtype, label_prdtype) * inputs.size(0)
                running_corrects_weight += calculate_accuracy(outputs_weight, label_weight) * inputs.size(0)
                running_corrects_halal += calculate_accuracy(outputs_halal, label_halal) * inputs.size(0)
                running_corrects_healthy += calculate_accuracy(outputs_healthy, label_healthy) * inputs.size(0)
                correct_preds_overall = ((outputs_prdtype.argmax(1) == label_prdtype) &
                                         (outputs_weight.argmax(1) == label_weight) &
                                         (outputs_halal.argmax(1) == label_halal) &
                                         (outputs_healthy.argmax(1) == label_healthy))
                running_corrects_overall += correct_preds_overall.sum().item()
                total_samples += inputs.size(0)

            epoch_loss = running_loss / total_samples
            epoch_acc_prdtype = running_corrects_prdtype / total_samples
            epoch_acc_weight = running_corrects_weight / total_samples
            epoch_acc_halal = running_corrects_halal / total_samples
            epoch_acc_healthy = running_corrects_healthy / total_samples
            epoch_acc_overall = running_corrects_overall / total_samples

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc_overall))

            if phase == 'val':
                if epoch_loss < best_val_loss:
                    print("new loss obtained")
                    best_val_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                    print(f"tmp epochs_no_improve: {epochs_no_improve}")

                if epochs_no_improve >= early_stopping_patience:
                    print("Early stopping triggered at epoch: {}".format(epoch + 1))
                    model.load_state_dict(best_model_wts)
                    return model, history
            
            print(f"epochs_no_improve: {epochs_no_improve}")

    model.load_state_dict(best_model_wts)
    return model, history

# Example usage of the function
# Assuming CONFIGS, trainLoader, testLoader, etc. are already defined
num_classes_prdtype = len(np.unique(trainLabels['labels_prdtype']))
num_classes_weight = len(np.unique(trainLabels['labels_weight']))
num_classes_halal = len(np.unique(trainLabels['labels_halal']))
num_classes_healthy = len(np.unique(trainLabels['labels_healthy']))

custom_resnet_model = MultiHeadResNet(num_classes_prdtype, num_classes_weight, num_classes_halal, num_classes_healthy)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
custom_resnet_model = custom_resnet_model.to(device)

criterion_prdtype = nn.CrossEntropyLoss()
criterion_weight = nn.CrossEntropyLoss()
criterion_halal = nn.CrossEntropyLoss()
criterion_healthy = nn.CrossEntropyLoss()

optimizer = optim.Adam(custom_resnet_model.parameters(), lr=CONFIGS['INIT_LR'])

criteria = (criterion_prdtype, criterion_weight, criterion_halal, criterion_healthy)

# Start time
print("Model training started...")
start_time = time.time()

model_ft, history = train_model(custom_resnet_model, criteria, optimizer, trainLoader, testLoader, device, num_epochs=CONFIGS['NUM_EPOCHS'])

# End time
end_time = time.time()
print("Model training completed...")

execution_time = end_time - start_time
print(f"Time spent: {round(execution_time/60,2)} mins")

In [ ]:
torch.save(model_ft.state_dict(), 'output/multi_head_model.pth')

print("[INFO] saving label encoder...")
f = open(CONFIGS["LE_PATH_PRDTYPE"], "wb")
f.write(pickle.dumps(le_prdtype))
f.close()
f = open(CONFIGS["LE_PATH_WEIGHT"], "wb")
f.write(pickle.dumps(le_weight))
f.close()
f = open(CONFIGS["LE_PATH_HALAL"], "wb")
f.write(pickle.dumps(le_halal))
f.close()
f = open(CONFIGS["LE_PATH_HEALTHY"], "wb")
f.write(pickle.dumps(le_healthy))
f.close()

In [15]:
def evaluate_model(model, data_loader, dataset_size, num_mc_samples=50):
    model.eval()  # Set the model to evaluation mode
    correct_counts = {'ProductType': 0, 'Weight': 0, 'HalalStatus': 0, 'HealthStatus': 0, 'Total': 0}
    total_distances = []

    with torch.no_grad():
        for (images, labels_prdtype, labels_weight, labels_halal, labels_healthy, filenames) in data_loader:
            images = images.to(CONFIGS['DEVICE'])
            labels_prdtype, labels_weight, labels_halal, labels_healthy = labels_prdtype.to(CONFIGS['DEVICE']), labels_weight.to(CONFIGS['DEVICE']), labels_halal.to(CONFIGS['DEVICE']), labels_healthy.to(CONFIGS['DEVICE'])

            # Forward pass
            out1, out2, out3, out4 = model(images)

            # Store deterministic predictions
            det_pred_prdtype = out1.argmax(1)
            det_pred_weight = out2.argmax(1)
            det_pred_halal = out3.argmax(1)
            det_pred_healthy = out4.argmax(1)

            # # Monte Carlo Dropout
            # mc_distances = []
            # model.train()  # Enable dropout
            # for i in range(num_mc_samples):
            #     mc_out1, mc_out2, mc_out3, mc_out4, _ = model(images)
            #     mc_pred_prdtype = mc_out1.argmax(1)
            #     mc_pred_weight = mc_out2.argmax(1)
            #     mc_pred_halal = mc_out3.argmax(1)
            #     mc_pred_healthy = mc_out4.argmax(1)

            #     distance = (det_pred_prdtype != mc_pred_prdtype).float() + \
            #                (det_pred_weight != mc_pred_weight).float() + \
            #                (det_pred_halal != mc_pred_halal).float() + \
            #                (det_pred_healthy != mc_pred_healthy).float()

            #     mc_distances.append(distance)

            # # Calculate average distance
            # avg_distance = torch.stack(mc_distances).mean(0)
            # total_distances.extend(avg_distance.cpu().numpy().tolist())

            # Restore to evaluation mode
            # model.eval()

            # Update correct counts for each category
            correct_counts['ProductType'] += (out1.argmax(1) == labels_prdtype).float().sum().item()
            correct_counts['Weight'] += (out2.argmax(1) == labels_weight).float().sum().item()
            correct_counts['HalalStatus'] += (out3.argmax(1) == labels_halal).float().sum().item()
            correct_counts['HealthStatus'] += (out4.argmax(1) == labels_healthy).float().sum().item()
            correct_counts['Total'] += ((out1.argmax(1) == labels_prdtype) & (out2.argmax(1) == labels_weight) & (out3.argmax(1) == labels_halal) & (out4.argmax(1) == labels_healthy)).float().sum().item()

    # Calculate accuracies
    accuracies = {key: correct_counts[key] / dataset_size for key in correct_counts}
    # avg_total_distance = sum(total_distances) / len(total_distances)

    # Plot histogram of distances
    # plt.hist(total_distances, bins=30, alpha=0.7, label='Total Distances', color='b')
    # plt.xlabel('Distance')
    # plt.ylabel('Frequency')
    # plt.title(f'Distribution of Total Distances: N={len(total_distances)}')
    # plt.show()

    # return accuracies, avg_total_distance, total_distances
    return accuracies

# Evaluate on training set
train_accuracies = evaluate_model(model_ft, trainLoader, len(trainDS))
print(f"Training Accuracies: {train_accuracies}")

# Evaluate on test set
test_accuracies= evaluate_model(model_ft, testLoader, len(testDS))
print(f"Test Accuracies: {test_accuracies}")


Training Accuracies: {'ProductType': 0.9893617021276596, 'Weight': 0.9851063829787234, 'HalalStatus': 0.9744680851063829, 'HealthStatus': 0.9872340425531915, 'Total': 0.9468085106382979}
Test Accuracies: {'ProductType': 0.6732673267326733, 'Weight': 0.5544554455445545, 'HalalStatus': 0.7524752475247525, 'HealthStatus': 0.8910891089108911, 'Total': 0.41089108910891087}


In [16]:
def evaluate_model2(model, data_loader, le_prdtype, le_weight, le_halal, le_healthy):
    model.eval()  # Set model to evaluation mode
    results = []

    with torch.no_grad():
        for (images, labels_prdtype, labels_weight, labels_halal, labels_healthy, filenames) in data_loader:
            images = images.to(CONFIGS['DEVICE'])
            out1, out2, out3, out4 = model(images)

            for idx in range(len(filenames)):
                correct_label = f"{le_prdtype.classes_[labels_prdtype[idx]]}_{le_weight.classes_[labels_weight[idx]]}_{le_halal.classes_[labels_halal[idx]]}_{le_healthy.classes_[labels_healthy[idx]]}"
                row = [filenames[idx], correct_label]
                row.extend(out1[idx].cpu().numpy())
                row.extend(out2[idx].cpu().numpy())
                row.extend(out3[idx].cpu().numpy())
                row.extend(out4[idx].cpu().numpy())
                results.append(row)

    # Define column names
    column_names = ['Filename', 'CorrectTotalLabel']
    column_names += ['ProductType_' + name for name in le_prdtype.classes_]
    column_names += ['Weight_' + name for name in le_weight.classes_]
    column_names += ['HalalStatus_' + name for name in le_halal.classes_]
    column_names += ['HealthStatus_' + name for name in le_healthy.classes_]

    # Create DataFrame
    results_df = pd.DataFrame(results, columns=column_names)
    return results_df


# Usage of the function
train_results_df = evaluate_model2(model_ft, trainLoader, le_prdtype, le_weight, le_halal, le_healthy)
test_results_df = evaluate_model2(model_ft, testLoader, le_prdtype, le_weight, le_halal, le_healthy)

# Concatenate the training and test results
combined_results_df = pd.concat([train_results_df, test_results_df], axis=0)
combined_results_df.reset_index(drop=True, inplace=True)

# Display the combined results
print("Combined Results:")
combined_results_df.head()


Combined Results:


Filename  \
0  IMG_3533_jpeg.rf.7c479d2b82aa319692d4c74ba4acf...   
1  IMG_2282_JPG.rf.55ff8559732da3aeccad420b17c3f9...   
2  IMG_20230428_123200_jpg.rf.34cc17cc4cb5a707327...   
3  Crackers_200-299g_0311_Halal_25_png.rf.c55545c...   
4  IMG_6362_jpeg.rf.16a8b744b3be105dc728d6b6bfefc...   

                                   CorrectTotalLabel  ProductType_AdultMilk  \
0                  Nuts_300-399g_NonHalal_NonHealthy              -2.493247   
1  BeehoonVermicelliMeesua_200-299g_Halal_NonHealthy               0.131873   
2                 Sugar_800-899g_NonHalal_NonHealthy              -2.153643   
3  BiscuitsCrackersCookies_200-299g_Halal_NonHealthy              -1.230329   
4         OtherBakingNeeds_900-999g_Halal_NonHealthy              -0.172884   

   ProductType_BabyMilkPowder  ProductType_Babyfood  \
0                   -2.458110             -0.752643   
1                    0.044445             -0.519548   
2                    2.140464             -0.861953   
3                   -2.523535              0.557129   
4                    1.589988             -0.664545   

   ProductType_BeehoonVermicelliMeesua  ProductType_BiscuitsCrackersCookies  \
0                             3.418245                             1.410686   
1                             7.003169                            -0.729551   
2                            -2.464264                             0.675603   
3                            -2.749804                             8.671530   
4                            -1.997344                            -0.707637   

   ProductType_Book  ProductType_BreakfastCerealsCornflakes  \
0         -1.484503                               -3.675712   
1         -3.285197                               -1.949823   
2         -1.984505                               -1.661865   
3         -4.183312                               -3.959238   
4         -3.679213                               -1.114295   

   ProductType_CannedPacketCreamersSweet  ...  Weight_400-499g  \
0                              -4.696049  ...         1.202736   
1                              -2.659935  ...        -0.359658   
2                              -4.919771  ...         1.070148   
3                              -3.233744  ...         0.550248   
4                              -2.144860  ...        -1.347105   

   Weight_500-599g  Weight_600-699g  Weight_700-799g  Weight_800-899g  \
0        -0.929754        -2.064444        -3.176098        -1.962588   
1         0.070565        -1.163762        -2.505035        -1.926098   
2        -1.785139        -3.493709        -2.736918         6.794210   
3        -2.701535        -0.062121        -1.760023        -0.467273   
4        -3.294689        -0.024379        -3.188744         1.041424   

   Weight_900-999g  HalalStatus_Halal  HalalStatus_NonHalal  \
0        -3.577293          -3.286956              2.287906   
1        -2.406361           3.796422             -2.279831   
2        -0.917141          -3.683269              4.121268   
3        -4.762103           3.189244             -1.663041   
4         6.573578           2.014626             -1.763547   

   HealthStatus_Healthy  HealthStatus_NonHealthy  
0             -4.930899                 2.953754  
1             -1.371966                 1.770485  
2             -4.709025                 1.865833  
3             -4.486520                 1.161967  
4             -4.737069                 4.561938  

[5 rows x 59 columns]

In [17]:
combined_results_df.to_csv('new_imgs_results_small_model.csv', index=False)

In [18]:
combined_results_df.shape

(672, 59)

In [13]:
import pandas as pd
import numpy as np
from collections import Counter

# Since the actual data is an image of text, we'll manually transcribe a few lines to demonstrate the process.
# In practice, the user would extract the text data using OCR (Optical Character Recognition) tools such as Tesseract.


# Convert the example data into a DataFrame
df = pd.read_csv("./results_[0.1].csv")

# Flatten the list of lists into a single list
all_indices = [i for sublist in df['Alpha Max Indices'] for i in sublist]

# Count the frequency of each number
frequency_counts = Counter(all_indices)

frequency_counts


Counter({'[': 1750,
         '3': 771,
         ',': 3500,
         ' ': 3500,
         '7': 736,
         ']': 1750,
         '6': 1142,
         '0': 650,
         '1': 740,
         '2': 840,
         '5': 360,
         '4': 11})